In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
train_dir = '/content/drive/MyDrive/Dataset_entrenamiento'
val_dir = '/content/drive/MyDrive/Dataset_validacion'

# Preprocesamiento y aumentación de datos
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Cargar los datos de las carpetas
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='binary')


In [ ]:
# Cargar VGG16 con pesos preentrenados en ImageNet, excluyendo la parte de clasificación
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Añadir capas finales para la clasificación binaria
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Capa global de pooling
x = Dense(1024, activation='relu')(x)  # Capa densa adicional
predictions = Dense(1, activation='sigmoid')(x)  # Capa de salida para clasificación binaria

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Congelar las capas del modelo base (opcional para transfer learning)
for layer in base_model.layers:
    layer.trainable = False

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Entrenamiento del modelo
history = model.fit(train_generator, epochs=10, validation_data=val_generator)


In [ ]:
# Evaluar el modelo con datos de validación
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred = np.where(y_pred > 0.5, 1, 0)  # Convertir probabilidades en clases binarias
y_true = val_generator.classes

# Métricas de Scikit-learn
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

# Métricas adicionales
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# Curva ROC y AUC
fpr, tpr, thresholds = roc_curve(y_true, model.predict(val_generator))
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Función para calcular el Dice Coefficient
def dice_coefficient(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (np.sum(y_true_f) + np.sum(y_pred_f))

# Calcular el Dice Coefficient para la validación
dice = dice_coefficient(y_true, y_pred)
print(f'Dice Coefficient: {dice}')
